In [ ]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time

# Initialize geocoder
geolocator = Nominatim(user_agent="restaurant_mapper")

# Function to convert address to longitude and latitude with a retry mechanism - handles timeouts and service errors
def geocode_address(address, max_retries=3):
    """Convert address to coordinates with retry logic""" # This names the function so you can refer to it later
    for attempt in range(max_retries):
        try:
            time.sleep(1)  # Respect rate limits of the geocoding service
            location = geolocator.geocode(address)
            if location:
                return location.latitude, location.longitude # Return lat and long if found
            return None, None
        except (GeocoderTimedOut, GeocoderServiceError):
            if attempt == max_retries - 1:
                return None, None
            time.sleep(2)
    return None, None

# Functions to determine marker color and icon
def get_marker_color(category):
    """Assign colors based on category"""
    colors = {
        'Food': 'red',
        'Nightlife': 'purple',
        'Activities': 'blue'
    }
    return colors.get(category, 'gray')

# Function to determine marker icon based on favorite status
def get_marker_icon(is_favorite):
    """Assign icon based on favorite status"""
    return 'star' if is_favorite else 'circle'

# Main function to create map
def create_map(csv_file, output_name, center_coords):
    """Create a map from CSV file"""
    # Read CSV
    df = pd.read_csv(csv_file)
    
    # Create map centered on the city with grayscale style
    m = folium.Map(
        location=center_coords, 
        zoom_start=12,
        tiles='https://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png', # Grayscale tiles from CARTO
        attr='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>' # Attribution for grayscale tiles
    )
    
    # Add markers for each location
    for idx, row in df.iterrows():
        print(f"Processing: {row['Name']}") # Log progress since not all addresses may geocode successfully
        
        # Geocode address
        lat, lon = geocode_address(row['Address'])
        
        if lat and lon:
            # Check if favorite
            is_favorite = str(row.get('Favorite', '')).strip().upper() == 'X'
            
            # Create marker
            folium.Marker(
                location=[lat, lon],
                popup=folium.Popup(
                    f"<b>{row['Name']}</b><br>"
                    f"{row['Category']}<br>"
                    f"{row['Address']}"
                    f"{'<br>⭐ FAVORITE' if is_favorite else ''}",
                    max_width=250
                ),
                tooltip=row['Name'],
                icon=folium.Icon(
                    color=get_marker_color(row['Category']),
                    icon=get_marker_icon(is_favorite),
                    prefix='fa'
                )
            ).add_to(m)
        else:
            print(f"   Could not geocode: {row['Name']}") # So you know which ones failed
    
    # Add legend in the bottom right corner
    legend_html = '''
    <div style="position: fixed; 
                bottom: 20px; right: 20px; 
                background-color: white; 
                border: 2px solid grey; 
                border-radius: 5px;
                padding: 10px; 
                font-size: 14px;
                font-family: Arial, sans-serif;
                z-index: 9999;
                box-shadow: 2px 2px 6px rgba(0,0,0,0.3);">
        <div style="margin-bottom: 8px; font-weight: bold;">Legend</div>
        <div style="margin-bottom: 5px;">
            <i class="fa fa-circle" style="color: red;"></i> Food
        </div>
        <div style="margin-bottom: 5px;">
            <i class="fa fa-circle" style="color: purple;"></i> Nightlife
        </div>
        <div style="margin-bottom: 5px;">
            <i class="fa fa-circle" style="color: blue;"></i> Activities
        </div>
        <div style="margin-top: 10px; padding-top: 8px; border-top: 1px solid #ccc;">
            <i class="fa fa-star" style="color: gold;"></i> Favorite
        </div>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))
    
    # Save map
    m.save(output_name)
    print(f"Map saved as {output_name}\n")

# City center coordinates
cities = {
    'Chicago': [41.8781, -87.6298],
    'LA': [34.0522, -118.2437],
    'NYC': [40.7128, -74.0060]
}

# Create maps for each city
create_map('data/Recs_toMap - Chicago.csv', 'chicago_map.html', cities['Chicago'])
create_map('data/Recs_toMap - LA.csv', 'la_map.html', cities['LA'])
create_map('data/Recs_toMap - NYC.csv', 'nyc_map.html', cities['NYC'])

Creating Chicago map...
Processing: Batter & Berries
Processing: Sunda
Processing: Pizzeria Portofino
Processing: The Art of Pizza
Processing: Lil' Ba-Ba-Reeba!
Processing: Quartino Ristorante
Processing: Zombie Tacos 
  ⚠️  Could not geocode: Zombie Tacos 
Processing: Los Comales
Processing: Qing Xiang Yuan Dumplings
Processing: Kasama
Processing: Oberweis
Processing: Ann Sathers
Processing: Mario's Italian Lemonade
Processing: Pearl Club
Processing: Three dots and a Dash
Processing: Punch Bowl Social
Processing: Emporium
Processing: The Bassment
Processing: Moe's
Processing: The Mine
Processing: The Art Institute of Chicago
Processing: The Village Discount 
Processing: River Walk
✓ Map saved as chicago_map.html

Creating LA map...
Processing: Hangari Kalguksu
Processing: Sun Nong Dan
Processing: Brother's Cousins
Processing: Pine and Crane
Processing: Fat Sal's 
Processing: NBC Seafood
Processing: Lunasia
Processing: Banh Mi My Tho
Processing: 626 Ice cream
Processing: Top Restaurant